# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,279.35,66,98,1.79,US,1700589708
1,1,edinburgh of the seven seas,-37.0676,-12.3116,286.58,79,7,17.09,SH,1700589708
2,2,iqaluit,63.7506,-68.5145,258.47,95,70,1.95,CA,1700589708
3,3,thompson,55.7435,-97.8558,264.57,75,99,5.90,CA,1700589708
4,4,klyuchi,52.2667,79.1667,264.83,78,10,5.13,RU,1700589708


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
fig = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    colormap='viridis',
    title='City Map with Humidity',
    hover_cols=['City', 'Country', 'Humidity'],
    size='Humidity',
    frame_width=600,
    frame_height=400,
    tiles='CartoLight',
    legend=True,
)

# Display the map
fig

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 10) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null 
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,mandi bahauddin,32.5834,73.4843,290.73,31,0,1.42,PK,1700589709
29,29,captainganj,26.9267,83.7153,292.30,42,0,2.86,IN,1700589711
34,34,tazacorte,28.6290,-17.9293,301.69,21,0,7.20,ES,1700589712
44,44,fortuna,40.5982,-124.1573,286.34,87,0,2.57,US,1700589713
55,55,sinjar,36.3209,41.8754,283.25,50,0,1.73,IQ,1700589715


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
7,mandi bahauddin,PK,32.5834,73.4843,31,
29,captainganj,IN,26.9267,83.7153,42,
34,tazacorte,ES,28.6290,-17.9293,21,
44,fortuna,US,40.5982,-124.1573,87,
55,sinjar,IQ,36.3209,41.8754,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "hotel",
    "radius": radius,
    "limit": 1,
    "sort": "distance",
    "apiKey": "7KfCeIxEghcQNocAfVOX"  # Make sure to replace with your actual API key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{row['Lng']},{row['Lat']}"
    params["bias"] = f"circle:{radius}@{row['Lng']},{row['Lat']}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mandi bahauddin - nearest hotel: No hotel found
captainganj - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
sinjar - nearest hotel: No hotel found
alesund - nearest hotel: No hotel found
kermit - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
burang - nearest hotel: No hotel found
santa ana - nearest hotel: No hotel found
hammerfest - nearest hotel: No hotel found
alta - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
sakaiminato - nearest hotel: No hotel found
aral - nearest hotel: No hotel found
pavlodar - nearest hotel: No hotel found
bluffdale - nearest hotel: No hotel found
lecheng - nearest hotel: No hotel found
nuku'alofa - nearest hotel: No hotel found
najafabad - nearest hotel: No hotel found
teguise - nearest hotel: No hotel found
sinan - nearest hotel: No hotel found
guilin - nearest hotel: No ho

,City,Country,Lat,Lng,Humidity,Hotel Name
7,mandi bahauddin,PK,32.5834,73.4843,31,No hotel found
29,captainganj,IN,26.9267,83.7153,42,No hotel found
34,tazacorte,ES,28.6290,-17.9293,21,No hotel found
44,fortuna,US,40.5982,-124.1573,87,No hotel found
55,sinjar,IQ,36.3209,41.8754,50,No hotel found
67,alesund,NO,62.4723,6.1549,67,No hotel found
89,kermit,US,31.8576,-103.0927,35,No hotel found
104,tsiombe,MG,-25.3000,45.4833,75,No hotel found
107,burang,CN,30.2500,81.1667,29,No hotel found
112,santa ana,US,33.7456,-117.8678,19,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
scatter_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    colormap='viridis',
    size=20,
    title='City Map with Hotel Information',
    xlabel='Longitude',
    ylabel='Latitude',
    colorbar=True,
    alpha=0.8,
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)